# 可視化用ノートブック

In [1]:
# ライブラリの読み込み
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [3]:
train.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,customer_info,ProdTaken
0,0,50歳,Self Enquiry,2,900秒,Large Business,male,1.0,4.0,Basic,3.0,5,1,4,Executive,253905.0,未婚 車未所持 子供なし,1
1,1,56歳,Company Invited,1,14分,Salaried,Male,1.0,4.0,Standard,3.0,2,1,4,Senior Manager,404475.0,"離婚済み,車あり,子供無し",0
2,2,NaN,Self Enquiry,1,10分,Large Business,Female,1.0,3.0,Basic,3.0,4,0,4,Executive,278145.0,"結婚済み、自動車未所有,子供なし",1
3,3,三十七歳,Self Enquiry,2,1080秒,Small Business,female,1.0,3.0,Standard,4.0,1,0,5,Senior Manager,326805.0,離婚済み、車所持、子供無し,0
4,4,48歳,Company Invited,3,1020秒,Small Business,female,1.0,3.0,Basic,4.0,4,0,4,Executive,258435.0,独身／車所持／無子,1


In [4]:
train.isnull().sum()

id                          0
Age                       100
TypeofContact               6
CityTier                    0
DurationOfPitch           121
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups          33
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips              22
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              56
customer_info               0
ProdTaken                   0
dtype: int64

In [5]:
train_dropped = train.dropna()
train_dropped.shape

(3206, 18)

In [6]:
# 前処理後のデータを格納するdataframe
train_preprocessed = pd.DataFrame()

In [7]:
match_type = r'[0-9]+|[一二三四五六七八九]+|[１２３４５６７８９]+'
kanji_to_arabic = str.maketrans("一二三四五六七八九１２３４５６７８９", "123456789123456789")

def extract_age(age: str) -> int:
    def kanji_to_int(kanji: str) -> int:
        matches: list[str] = re.findall(match_type, age)
        if len(matches) == 0:
            return -1
        else:
            ret = 0
            keta = 0
            for match in matches:
                str_age = match.translate(kanji_to_arabic)
                numeric_age = int(str_age)
                if keta == 0:
                    ret = numeric_age
                    keta += len(match)
                elif keta == 1:
                    ret = ret*10 + numeric_age
                    keta += len(match)
                elif keta == 2:
                    ret = ret*100 + numeric_age
                    keta += len(match)
                else:
                    return -1
            return ret
    if age.find("代") != -1:
        dai_int = kanji_to_int(age)
        return dai_int + 5
    else:
        return kanji_to_int(age)

age_int = train_dropped['Age'].apply(extract_age)
train_dropped.loc[:, 'Age_int'] = age_int
# 保存
train_dropped.to_csv("../output/train_dropped.csv", index=False)

train_preprocessed = train_dropped[['Age_int']]
train_preprocessed.head()


/tmp/ipykernel_54634/3215462457.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dropped.loc[:, 'Age_int'] = age_int


,Age_int
0,50
1,56
3,37
4,48
5,19


In [8]:
train_dropped["TypeofContact"].value_counts()

TypeofContact
Self Enquiry       2050
Company Invited    1156
Name: count, dtype: int64